In [ ]:
#First step - Set up our code

import os, arcpy
from datetime import datetime

os.chdir(r"C:\Users\secuc\ArcGIS_Projects\Project")
print(f"{os.getcwd()}")

mosaicData = "MODIS_2023"
mosaicResam = f"C:\\Users\\secuc\\MyFiles\\Resampled\\{mosaicData}_Resample.crf"
mosaicResam_1 = f"C:\\Users\\secuc\\MyFiles\\Resampled_27800\\{mosaicData}_Resample27.crf"
rasterfolder = r"C:\Users\secuc\MyFiles\MODIS_Download\MODIS_2023"
aGDB = r"C:\Users\secuc\ArcGIS_Projects\Project\MODIS.gdb"

print(f'{mosaicData}\nDone at {datetime.now().strftime("%H:%M:%S")}!')

In [ ]:
#Second Step - Set up parameters & run Create Mosaic.
#Third Step - Add Rasters Tools

updatecs = "NO_CELL_SIZES"
updatebnd = "UPDATE_BOUNDARY"
updateovr = "UPDATE_OVERVIEWS"
maxlevel = ""
maxcs = ""
maxdim = ""
spatialref = ""
inputdatafilter = "*.nc"
subfolder = "SUBFOLDERS"
duplicate = "EXCLUDE_DUPLICATES"
buildpy = "NO_PYRAMIDS"
calcstats = "CALCULATE_STATISTICS"
buildthumb = "NO_THUMBNAILS"
comments = "Add Raster Datasets"
forcesr = ""
estimatestats = "ESTIMATE_STATISTICS"
auxilaryinput = ""
enablepixcache = "NO_PIXEL_CACHE"
cachelocation = ""

arcpy.CreateMosaicDataset_management(f"{aGDB}", f"{mosaicData}", arcpy.SpatialReference(3857))

arcpy.management.AddRastersToMosaicDataset(f"{aGDB}\\{mosaicData}", "NetCDF", f"{rasterfolder}", f"{updatecs}", f"{updatebnd}", f"{updateovr}", f"{maxlevel}", f"{maxcs}", f"{maxdim}", f"{spatialref}", f"{inputdatafilter}", f"{subfolder}", f"{duplicate}", f"{buildpy}", f"{calcstats}", f"{buildthumb}", f"{comments}", f"{forcesr}", f"{estimatestats}", f"{auxilaryinput}", f"{enablepixcache}", f"{cachelocation}")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

In [ ]:
#Fourth Step - Create and populate the field to extract the date from the Name. 
#Fifth convert to date.

arcpy.management.AddField(mosaicData, "Time", "TEXT")

with arcpy.da.UpdateCursor(mosaicData, ["Name", "Time"]) as aCursor:
    for aRow in aCursor:
        if "TERRA" in aRow[0]:
            aRow[1] = aRow[0][12:20]
            aCursor.updateRow(aRow)

arcpy.ConvertTimeField_management(f"{mosaicData}\Footprint", "Time", "yyyyMMddHHmmss", "Time_Extracted")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

### Sixth Step - Build the multidimensional information.

arcpy.md.BuildMultidimensionalInfo(f"{mosaicData}_2", "chlor_a", "Time_Extracted")

print("Done!")

### Seventh Step - Resample 4630 - UpSampling - higher resolutions but worse statistics.
### To prevent False Accuracy it is best to downsample

arcpy.management.Resample(mosaicData, mosaicResam, 4630, "NEAREST")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

In [ ]:
# 27800 meters ----- DownSampling - Lower Resolution but better statistics
mosaicData = ["MODIS_2011", "MODIS_2012", "MODIS_2013", "MODIS_2014", "MODIS_2015", "MODIS_2016", "MODIS_2017", "MODIS_2018", "MODIS_2019", "MODIS_2020", "MODIS_2021", "MODIS_2022", "MODIS_2023"]

for item in mosaicData:
    arcpy.management.Resample(item, f"C:\\Users\\secuc\\MyFiles\\Resampled_27800\\{item}_Resample27.crf", 27820, "NEAREST")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

In [ ]:
# Data managing...

arcpy.DeleteField_management(mosaicData, "Time")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')